In [13]:

import src
# TODO: Delete after development
import importlib
importlib.reload(src)

from src import *

In [14]:
configuration = Configuration()

In [15]:
authors = []
for author_name in FileUtils.read_authors(configuration.selected_authors_filepath):
    author = Author(name=author_name)
    author.read_book_collection(configuration.all_book_data_directory)
    author.read_generated_texts(configuration.model_dirs)
    authors.append(author)

In [16]:
analysis = Analysis(configuration.analysis_size)
analysis_data = analysis.analyze(authors)

In [17]:
AnalysisVisualization().visualize(analysis_data)